<a href="https://colab.research.google.com/github/jude1997felix/Decentralized-Air-quality-Monitoring-and-Prediction/blob/main/competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Packages

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.mlab as mlab
plt.style.use('ggplot')
from matplotlib.pyplot import figure
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MultiLabelBinarizer
import os
import datetime
import math 
import IPython
import IPython.display
import matplotlib as mpl
import tensorflow as tf

In [29]:
def compile_and_fit(model, window, patience=3):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history



# def compile_and_fit(model, window, patience=2, recompile=True):
#   early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
#                                                     patience=patience,
#                                                     mode='min')

#   def smape(y_true, y_pred):
#     return tf.reduce_mean(2 * tf.abs(y_true - y_pred)
#                           / (tf.abs(y_pred) + tf.abs(y_true)) , axis=-1)

#   if recompile:
#       model.compile(loss=tf.losses.MeanSquaredError(),
#                     optimizer=tf.optimizers.Adam(),
#                     metrics=[smape])
#                 # metrics=[tf.metrics.MeanAbsoluteError()])

#   history = model.fit(window.train, epochs=MAX_EPOCHS,
#                       validation_data=window.val,
#                       callbacks=[early_stopping])
#                       # )
#   return history

In [19]:
def plot(self, model=None, plot_col='PM10 (107)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(12, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')


In [20]:
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds


def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels




In [21]:
df = pd.read_csv('/content/imputed_csv_8779.csv',parse_dates=['Start'])
df.head()

,Unnamed: 0,Start,Slut,NO2 (183),NOX as NO2 (184),PM10 (182),PM2.5 (181)
0,0,2015-01-01 00:00:00,2015-01-01 01:00:00,33.3,69.5,101.2,51.9
1,1,2015-01-01 01:00:00,2015-01-01 02:00:00,26.1,39.0,12.5,9.3
2,2,2015-01-01 02:00:00,2015-01-01 03:00:00,18.4,25.7,12.2,9.2
3,3,2015-01-01 03:00:00,2015-01-01 04:00:00,18.1,33.5,16.2,8.9
4,4,2015-01-01 04:00:00,2015-01-01 05:00:00,11.0,22.1,14.5,5.9


In [22]:
del df['Unnamed: 0']

In [23]:
df.head()

,Start,Slut,NO2 (183),NOX as NO2 (184),PM10 (182),PM2.5 (181)
0,2015-01-01 00:00:00,2015-01-01 01:00:00,33.3,69.5,101.2,51.9
1,2015-01-01 01:00:00,2015-01-01 02:00:00,26.1,39.0,12.5,9.3
2,2015-01-01 02:00:00,2015-01-01 03:00:00,18.4,25.7,12.2,9.2
3,2015-01-01 03:00:00,2015-01-01 04:00:00,18.1,33.5,16.2,8.9
4,2015-01-01 04:00:00,2015-01-01 05:00:00,11.0,22.1,14.5,5.9


In [24]:
  del df['Start']
  del df['Slut']
  column_indices = {name: i for i, name in enumerate(df.columns)}

  n = len(df)

  cols = df.columns
  # x = df.values #returns a numpy array
  sc = StandardScaler()
  # x_scaled = sc.fit_transform(x)
  # df = pd.DataFrame(x_scaled)

  # y = pm10_test.values
  # y_scaled = sc.fit_transform(y)
  # pm10_test = 
  
  df.columns = cols
  # Normalising training data
  train_df = df[0:int(n*0.7)]
  x = train_df.values
  train_scaled = sc.fit_transform(x)
  train_df = pd.DataFrame(train_scaled,columns=cols)
  # Normalising validation data
  val_df = df[int(n*0.70):int(n*0.85)]
  x = val_df.values
  train_scaled = sc.fit_transform(x)
  val_df = pd.DataFrame(train_scaled,columns=cols)
  # Normalising testing data
  test_df = df[int(n*0.85):]
  x = test_df.values
  train_scaled = sc.fit_transform(x)
  test_df = pd.DataFrame(train_scaled,columns=cols)


  num_features = df.shape[1]
  # return(train_df,val_df,test_df,num_features)
  train_mean = train_df.mean()
  train_std = train_df.std()

  df_std = (df - train_mean) / train_std

In [25]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
                train_df=train_df, val_df=val_df, test_df=test_df,
                label_columns=None):
      # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

      # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                      enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                            enumerate(train_df.columns)}

      # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])


WindowGenerator.split_window = split_window
WindowGenerator.plot = plot
WindowGenerator.make_dataset = make_dataset

@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
      # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
      # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

  

# single_step_window = WindowGenerator(
#       input_width=2, label_width=1, shift=1,
#       label_columns=['PM10 (107)'])
#   # single_step_window
# print(single_step_window.test)


In [26]:
OUT_STEPS = 24
multi_window = WindowGenerator(input_width=24,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS)

# multi_window.plot()
multi_window

Total window size: 48
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
Label indices: [24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
Label column name(s): None

In [27]:
MAX_EPOCHS = 20
multi_dense_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
    # Shape => [batch, 1, dense_units]
    tf.keras.layers.Dense(512, activation='relu'),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

history = compile_and_fit(multi_dense_model, multi_window)

# IPython.display.clear_output()
# multi_val_performance['Dense'] = multi_dense_model.evaluate(multi_window.val)
multi_performance = multi_dense_model.predict(multi_window.test)
# multi_window.plot(multi_dense_model)

Epoch 1/20
958/958 [==============================] - 4s 5ms/step - loss: 0.8339 - mean_absolute_error: 0.6308 - val_loss: 0.7896 - val_mean_absolute_error: 0.6200
Epoch 2/20
958/958 [==============================] - 4s 4ms/step - loss: 0.8254 - mean_absolute_error: 0.6269 - val_loss: 0.7896 - val_mean_absolute_error: 0.6206
Epoch 3/20
958/958 [==============================] - 4s 4ms/step - loss: 0.8222 - mean_absolute_error: 0.6257 - val_loss: 0.7899 - val_mean_absolute_error: 0.6195


In [28]:
multi_performance.shape

(6527, 24, 4)

In [37]:
no2 = multi_performance[0:10][0:][0]

In [38]:
no2.shape

(24, 4)

In [39]:
no2

array([[-0.9558216 , -0.8205393 , -0.01303597, -0.0851728 ],
       [-0.80753076, -0.7281708 ,  0.01104225, -0.07467635],
       [-0.69369125, -0.6105226 ,  0.04309293, -0.04100066],
       [-0.5885495 , -0.5137566 ,  0.10517125, -0.02029038],
       [-0.5531428 , -0.49779776,  0.17209256, -0.02989908],
       [-0.5070606 , -0.45167074,  0.21165392, -0.03693254],
       [-0.4677767 , -0.4673772 ,  0.25864995, -0.02205933],
       [-0.4116676 , -0.437586  ,  0.27430007, -0.05097131],
       [-0.39523262, -0.43812338,  0.33533707, -0.01970076],
       [-0.37876835, -0.42617124,  0.31121847, -0.02273868],
       [-0.31101286, -0.3778787 ,  0.39379403,  0.00889403],
       [-0.25829226, -0.3263988 ,  0.38733295, -0.00249196],
       [-0.24436893, -0.33851844,  0.40744594,  0.05433448],
       [-0.26544398, -0.35570925,  0.36551923,  0.09099661],
       [-0.27145833, -0.3571392 ,  0.3228343 ,  0.10616386],
       [-0.30971578, -0.38144356,  0.30538318,  0.13270882],
       [-0.30957785, -0.